# GETTIN
## Planejamento de Equipamentos Escolares
## Método: Capacitaded Facility Location
### Versão: 0.9
#### Fillipe O Feitosa <fillipefeitosa@ua.pt>

---


#### Math Programming


\begin{split}& \text{minimizar} \quad    & \sum_{j=1}^m f_j x_j +\sum_{i=1}^n \sum_{j=1}^m c_{ij} y_{ij} &     \\
& \text{sujeito a:} \quad & \sum_{j=1}^m y_{ij} =d_i  &  \mbox{ for }  i=1,\cdots,n \\
&    & \sum_{i=1}^n y_{ij} \leq M_j x_j & \mbox{ para } j=1,\cdots,m  \\
&    & y_{ij} \leq d_i x_j              & \mbox{ para } i=1,\cdots,n; j=1,\cdots,m \\
&    & y_{ij} \geq 0                    & \mbox{ para } i=1,\cdots,n; j=1,\cdots,m \\
&    & x_j \in \{ 0,1 \}                & \mbox{ para } j=1,\cdots,m\end{split}

#### List of Imports

In [1]:
# Import Libraries
from gurobi import *
import math
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import geopy.distance
from collections import defaultdict
# TEST Used for testing with a small sample
import random


#### Problem Data

In [2]:
# ------------------------- Problem data ------------------------- #

# Dict for SubSections With Demand
rawData = pd.read_excel('./subseccao_vagos/Pop_escolar_Vagos.xlsx')
listOfSubsections_2010 = {}
for e in range(len(rawData)-1):
    listOfSubsections_2010[int(rawData.SUBSECCAO[e])] = rawData.P_esc_1CEB_2010[e]+ rawData.P_esc_2CEB_2010[e]+rawData.P_esc_3CEB_2010[e]+rawData.P_esc_secund_2010[e]+rawData.Pre_escolar_2010[e]

    
# TEST Used _sample for testing with a small sample to CREATE A SAMPLE OF POTENTIAL SCHOOL POINTS   
# listOfSubsections_2010_sample = {}

# for i in range(80):
#     sampleSchool, sampleDemand = random.choice(list(listOfSubsections_2010.items()))
#     # If statement: Do not repeat random schools
#     if sampleSchool not in listOfSubsections_2010_sample.keys():
#         listOfSubsections_2010_sample[sampleSchool] = sampleDemand
        

# Dict of Lists for Schools with Capacity And Cost by Size
tupleOfCapacity = (80, 120, 200)

# TEST used for testing with small sample
# tupleOfCapacity = (10, 20, 50)

schoolsCapacityCost = defaultdict(list)

# # TEST Using _SAMPLE to get a sample of Points to create schools
for bgri in list(listOfSubsections_2010.keys()):
    for capacity in tupleOfCapacity:
        cost = capacity*6200
        tempList = (capacity, cost)
        schoolsCapacityCost[bgri].append(tempList)


# Retrieve Coordinates and Vinculate to Identifier (BGRI)
vagos = gpd.read_file('./data_gettin/vagos.geojson');
centroids = vagos.centroid

iteratorHandler = centroids.size
centroidVector = []
for centroid in centroids:
    obj = [centroid.xy[0][0], centroid.xy[1][0]]
    centroidVector.append(obj)
    
coordinates_X = {}
coordinates_Y = {}
for e in range (len(vagos)):
    coordinates_X[int(vagos.BGRI11[e])] = centroids[e].xy[0][0]
    coordinates_Y[int(vagos.BGRI11[e])] = centroids[e].xy[1][0]
    len
coordinates_X[list(listOfSubsections_2010.keys())[0]]

-8.69434885302111

In [3]:
# if 1180400403 not in listOfSubsections_2010_sample.keys():
#     print("nao está")
# sum(listOfSubsections_2010.values())

# schoolsCapacityCost


#### Modeling and creation of decision Variables

In [4]:
   
# -------- Distance Matrix Logic ------- #
transportationCosts = {}

for bgri_1 in list(listOfSubsections_2010.keys()):
    coords_1 = (coordinates_X[bgri_1], coordinates_Y[bgri_1])
    for bgri_2 in list(listOfSubsections_2010.keys()):
        coords_2 = (coordinates_X[bgri_2], coordinates_Y[bgri_2])
        # 0.29 cents per KM * 180 days
        cost = geopy.distance.vincenty(coords_1, coords_2).km*0.29*180*5
        transportationCosts[bgri_1, bgri_2] = cost
        
transportationCosts


{(1180100101, 1180100101): 0.0,
 (1180100101, 1180100102): 53.85232201410466,
 (1180100101, 1180100103): 75.70593525830172,
 (1180100101, 1180100104): 189.63754020712878,
 (1180100101, 1180100105): 124.33378869616459,
 (1180100101, 1180100106): 259.27879343375247,
 (1180100101, 1180100107): 322.6716804723019,
 (1180100101, 1180100108): 308.2463745258148,
 (1180100101, 1180100109): 424.6501600078313,
 (1180100101, 1180100110): 334.9931421300826,
 (1180100101, 1180100111): 375.2085475324842,
 (1180100101, 1180100112): 401.0625134054475,
 (1180100101, 1180100113): 380.2706810688313,
 (1180100101, 1180100114): 429.24666337842524,
 (1180100101, 1180100115): 572.6506288846633,
 (1180100101, 1180100116): 536.5653861795611,
 (1180100101, 1180100117): 189.1663824102672,
 (1180100101, 1180100118): 329.9827925739894,
 (1180100101, 1180100119): 270.9734316524683,
 (1180100101, 1180100120): 408.61721500237127,
 (1180100101, 1180100121): 274.19160105050986,
 (1180100101, 1180100122): 346.88097983249

In [5]:
# for school in list(schoolsCapacityCost.keys()):
#     for schoolSize in range(len(schoolsCapacityCost[school])):
#         print(schoolsCapacityCost[school][schoolSize][0])
#         print(schoolSize)

In [6]:

# -------- Decision Variables ---------- #

# numSchools = len(escola)
# numSubSections = len(subSecao)

# Creting Guroby Model
m = Model()

# Decision Variables
x = {}
y = {}
# d = {} # Distance Matrix
# @alpha: 0.29 de custo por Km  por (365 dias * 5 anos) 
# alpha = 529.25

# creating binary variable for every school
for j in list(schoolsCapacityCost.keys()):
    for schoolSize in tupleOfCapacity:
        x[(j,schoolSize)] = m.addVar(vtype=GRB.BINARY, name="(%d,%d)" % (j, schoolSize))

# creating continuous variable for subsections to check suply fractions
for subsection in list(listOfSubsections_2010.keys()):
    for school in list(schoolsCapacityCost.keys()):
        for capacity in schoolsCapacityCost[school]:
        # y of Subsection Suply
            y[(subsection,school,capacity[0])] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Fração da Sub[%d], escola[%d][%d]" % (subsection,school, capacity[0]))

m.update()

In [7]:
# schoolsCapacityCost[1180100101][1]
# listOfSubsections_2010[1180100101]
# for school in list(schoolsCapacityCost.keys()):
#     for schoolSize in range(len(schoolsCapacityCost[school])):
#         sizeOfSchool = schoolsCapacityCost[school][schoolSize][0]
#         print(x[school, sizeOfSchool])
# for capacity in schoolsCapacityCost[1180800306]:
#     print(capacity[0])

# for capacity in tupleOfCapacity:
#     print(capacity)
    
# for school in schoolsCapacityCost:
#     resultCache = [item for item in x if item[0] == school]
#     print(resultCache)
#     for e in resultCache:
#         print(str(e))

# for school in schoolsCapacityCost.keys():
#     print(x[(school, k)] for k in tupleOfCapacity)
#     m.addConstr(quicksum)

# for element in x:
#     print(element)

# len(x[(1180700102, 120)])
# m.getVars()
# for school in schoolsCapacityCost:
#     resultCache = [item for item in x if item[0] == school]
# #     m.addConstr( <= 1, "SameSiteRestriction(%s)"%(resultCache))
#     print(x[resultCache[1]])


# for i in range(len(resultCache)):
#     print(x[resultCache[i]])

# resultado = x[resultCache[0]]+x[resultCache[1]] + x[resultCache[2]]
# print(resultado)

## Adding Constraints

In [8]:
# Constraint for Every Student on School
# for i in range(numSubSections):
#     m.addConstr(quicksum(y[(i,j)] for j in range(numSchools)) == 1)

# Constraint to repect demands on Every Subsection - OK
for i in list(listOfSubsections_2010.keys()):
    m.addConstr(quicksum(y[(i,j,k)] for j in list(schoolsCapacityCost.keys()) for k in tupleOfCapacity) == listOfSubsections_2010[i])
    
# Constraint to restrain school capacity - OK
for school in list(schoolsCapacityCost.keys()):
    for schoolSize in range(len(schoolsCapacityCost[school])):
        numericalSizeOfSchool = schoolsCapacityCost[school][schoolSize][0]
        m.addConstr(quicksum(y[(i,school, numericalSizeOfSchool)] for i in list(listOfSubsections_2010.keys())) <= numericalSizeOfSchool * x[school, numericalSizeOfSchool], "SchoolK(%s,%s)"%(school, numericalSizeOfSchool))
    
# Add restriction to fraction Size - OK
for (i,j,k) in y:
    m.addConstr(y[i,j,k] <= listOfSubsections_2010[i]*x[j, k], "FractionConst(%s,%s,%s)"%(i,j,k))

# Restriction to exclude multiple schools being constructed on the same site
for school in schoolsCapacityCost:
    resultCache = [item for item in x if item[0] == school]
    # HardCoded The Sum of Results to That Group of Schools Range(3) [0,1,2]
    cacheQuickSum = x[resultCache[0]]+x[resultCache[1]]+x[resultCache[2]]
    m.addConstr( cacheQuickSum <= 1, "SameSiteRestriction(%s)"%(cacheQuickSum))

#### Objetive


In [9]:
# Setting objective

m.setObjective(
    quicksum(schoolsCapacityCost[school][schoolSize][1]*x[school, schoolsCapacityCost[school][schoolSize][0]] for school in list(schoolsCapacityCost.keys()) for schoolSize in range(len(schoolsCapacityCost[school]))) + 
    quicksum(transportationCosts[i,j]*y[i,j,k] for i in list(listOfSubsections_2010.keys()) for j in list(schoolsCapacityCost.keys()) for k in tupleOfCapacity), GRB.MINIMIZE )



In [ ]:
m.optimize()

Optimize a model with 1224322 rows, 1223046 columns and 4790742 nonzeros
Variable types: 1221132 continuous, 1914 integer (1914 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+02]
  Objective range  [8e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 5e+01]
Presolve removed 97665 rows and 97614 columns (presolve time = 5s) ...
Presolve removed 97665 rows and 97614 columns (presolve time = 12s) ...
Presolve removed 97665 rows and 97614 columns
Presolve time: 12.60s
Presolved: 1126657 rows, 1125432 columns, 4497900 nonzeros
Variable types: 1123518 continuous, 1914 integer (1914 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolved: 1126657 rows, 1125432 columns, 4497900 nonzeros

Root barrier log...

Ordering time: 0.62s

Barrier statistics:
 Dense cols : 1914
 AA' NZ     : 4.498e+06
 Factor NZ  : 1.424e+07 (roughly 1.0 GBytes of memory)
 Factor Ops : 2.871e+10 (roughly 6 seco

In [ ]:
m.getConstrs()

In [ ]:
print('Obj: %g' % m.objVal)

In [ ]:
resultsBgriCapacity = []
resultsSubFractionToSchool = []
for v in m.getVars():
    if(v.x != 0):
        print('%s   %g' % (v.varName, v.x))
        if(v.varName.startswith('(')):
            resultsBgriCapacity.append(v.varName)        
        else:
            resultsSubFractionToSchool.append(v.varName)

In [ ]:
len(resultsBgriCapacity)

In [ ]:
print(resultsBgriCapacity)

In [ ]:
m = None

In [ ]:
disposeDefaultEnv()

# 